# Get pdf

In [1]:
from pathlib import Path
ROOT_DIR = Path().resolve().parent

pdf_path = ROOT_DIR / 'data' / 'pure_pdfs' / "ISS.pdf"

# Extract text

In [2]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        extracted_text = ""
        
        # Iterate over all pages in the PDF
        for page in pdf.pages:
            extracted_text += page.extract_text()
        
    return extracted_text

text = extract_text_from_pdf(pdf_path)
print("Extracted Text:\n", text)

Extracted Text:
 INTEGRATED SERVICE SOLUTIONS
Invoice To:
Direct Produce Supplies LTD
57-63 Church Street
Wimbledon
SW19 5DQ
Invoice Date: 22/11/2023 Invoice No:22806
Quantity Service Description Rate (£) Net Value (£)
178,082 ISS SP - Picking: 12/11/2023 - 18/11/2023 Teynham 0.090 10,641.96
105,084 ISS SP - Picking: 12/11/2023 - 18/11/2023 Linton 0.090 2,856.15
36,065 ISS SP - Picking: 12/11/2023 - 18/11/2023 Sittingbourne 0.090 1,762.74
Net Total (£): 15,260.85
VAT (£): 3,052.17
Bank Details: HSBC UK Bank Plc Payment Terms: 28 days from invoice date Gross Total (£): 18,313.02
Sort Code: 40-03-21
Account No: 2236 6789
Integrated Service Solutions Limited, C/O Fowler Welch, London Road, Teynham, Sittingbourne, ME9 9PR
VAT Registration No: 184 6959 49 Company Registration No: 08332191
Contact
T 01795 523 310
E accounts@issproduce.co.uk
W www.issproduce.co.uk


# Using very small model due to collab limitations

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


print("Loading model and tokenizer...")
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model = AutoModelForCausalLM.from_pretrained(model_name, use_cache=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_cache=True)
print("Loaded model and tokenizer")

# reducing model's parameters size
model = model.half()

Loading model and tokenizer...
Loaded model and tokenizer


In [4]:
from jsonformer.main import Jsonformer
from jsonformer.format import highlight_values
import sys
sys.path.append('..')
from configs.configs import invoice_schema, get_prompt

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=invoice_schema,
    prompt=get_prompt(text),
    temperature=0.05,
    max_array_length=100,
    max_number_tokens=3072,
    max_string_token_length=80
)

print("Generating...")
output = builder()

highlight_values(output)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating...
{
  invoice_number: "22806",
  invoice_date: "22/11/2023",
  supplier_name: "Direct Produce Supplies LTD",
  customer_name: "Integrated Service Solutions Limited",
  total_net_amount: "15,260.85",
  total_vat_amount: "3,052.17",
  total_invoice_amount: "18,313.02",
  line_items: [
    {
      product_code: "ISS SP - Picking",
      description: "Service Description",
      quantity: "178,082",
      price_per_unit: "0.090",
      vat_percent: "9.0%",
      net_price: "10,641.96"
    },
    {
      product_code: "ISS SP - Picking",
      description: "Service Description",
      quantity: "105,084",
      price_per_unit: "0.090",
      vat_percent: "9.0%",
      net_price: "2,856.15"
    }
  ],
  due_date: "22/11/2023",
  payment_date: "22/11/2023",
  currency: "GBP",
  payment_details: {
    iban: "HSBC UK Bank Plc",
    swift: "40-03-21",
    account_number: "2236 6789"
  }
}


In [5]:
from json2html import json2html
from IPython.display import display, HTML
response = json2html.convert(output)
display(HTML(response))